In [1]:
%load_ext autoreload
%autoreload 2

# Takeaway, when diagonalising an array of matrices, the loop overhead is negligible compared to the actual diagonalisations, stick to python, do a get a 2x speedup from parrellism but that doesn't heko for the jobs limited to one core

In [2]:
import numpy as np
from disorder_model import correlated_noise
from FKMC.general import interaction_matrix
from munch import Munch
from scipy.linalg import eigh_tridiagonal
from scipy.linalg.lapack import dstev
import cython
%load_ext Cython

def solve_H(state, mu, beta, U, J_matrix, t, **kwargs):
    state = np.array(state)
    muf = muc = mu

    evals, evecs = eigh_tridiagonal(d = U*(state - 1/2) - muc, e =-t*np.ones(state.shape[0] - 1), lapack_driver = 'stev')
    Ff = - U/2*np.sum(state - 1/2) - muf*np.sum(state) + (state - 1/2).T @ J_matrix @ (state - 1/2)
    Fc = - 1/beta * np.sum(np.log(1 + np.exp(- beta * evals)))

    return Ff, Fc, evals, evecs


def solve_H_2(state, mu, beta, U, J_matrix, t, **kwargs):
    state = np.array(state)
    muf = muc = mu

    evals, evecs, info =  dstev(d = U*(state - 1/2) - muc, e =-t*np.ones(state.shape[0] - 1), compute_v = True)
    Ff = - U/2*np.sum(state - 1/2) - muf*np.sum(state) + (state - 1/2).T @ J_matrix @ (state - 1/2)
    Fc = - 1/beta * np.sum(np.log(1 + np.exp(- beta * evals)))

    return Ff, Fc, evals, evecs

repeats = 1000 #disorder realisations
N = 50 # system size
states = np.random.choice((0.0,1.0), size = (repeats, N))
params = Munch(mu=0.0, beta = 0.5, U = 5.0, t = 1.0, N = N, alpha = 1.5, J = 1.0)
params.J_matrix = interaction_matrix(**params)

%time Ffs_original = np.array([solve_H(s, **params)[0] for s in states])
%time Ffs_original = np.array([solve_H_2(s, **params)[0] for s in states])
#Ffs_vectorised = solve_H_vectorised(states, **params)

CPU times: user 1.1 s, sys: 16 ms, total: 1.11 s
Wall time: 278 ms
CPU times: user 888 ms, sys: 12 ms, total: 900 ms
Wall time: 226 ms


## Result seems to be that nothing really helps much, maybe use dstev instead of tridiagonal but cythonising the loop doesn't help

In [4]:
from FKMC.general import solve_H as solve_H_from_FKMC, shapes

def solve_H_vectorised(states, mu, beta, U, J_matrix, t, **kwargs):
    R, N = states.shape# = (disorder realisations, system size)
    
    muf = muc = mu
    evals = np.empty(shape = states.shape, dtype = np.float64)
    evecs = np.empty(shape = states.shape + (states.shape[-1],), dtype = np.float64)
    e = -t*np.ones(states.shape[-1] - 1)
    ds = U*(states - 1/2) - muc
    
    for i in range(R):
        evals[i], evecs[i] = eigh_tridiagonal(d = ds[i], e = e, lapack_driver = 'stev')
    
    #the below is a vecorised version of:
    #Ff = - U/2*np.sum(state - 1/2) - muf*np.sum(state) + (state - 1/2).T @ J_matrix @ (state - 1/2)
    #Fc = - 1/beta * np.sum(np.log(1 + np.exp(- beta * evals)))
    
    S = np.sum(states, axis = -1)
    t = (states - 1/2)
    #shapes(t, J, t) = [(R, N), (N, N), (R, N)] giving contraction ij,ik,ik -> i
    inner_prod = np.einsum('ij,jk,ik -> i', t, J_matrix, t, optimize = 'greedy')
    
    Ff = - U/2*(S - N/2) - muf*S + inner_prod
    Fc = - 1/beta * np.sum(np.log(1 + np.exp(- beta * evals)), axis = -1)
    
    #shapes(Ff, Fc, evals, evecs) = [(R,), (R,), (R, N), (R, N, N)]
    return Ff, Fc, evals, evecs

nevals = np.array([solve_H_from_FKMC(state, **params)[2] for state in states])
nevecs = np.array([solve_H_from_FKMC(state, **params)[3] for state in states])

%time nFf, nFc = np.array([solve_H_from_FKMC(state, **params)[:2] for state in states]).T

%time Ff, Fc, evals, evecs = solve_H_vectorised(states, **params)

shapes(nFc, nFf, Ff, Fc, evals, evecs)

np.allclose(nevals, evals), np.allclose(nevecs, evecs), np.allclose(Ff, nFf), np.allclose(Fc, nFc)

CPU times: user 860 ms, sys: 4 ms, total: 864 ms
Wall time: 216 ms
CPU times: user 884 ms, sys: 8 ms, total: 892 ms
Wall time: 222 ms
[(1000,), (1000,), (1000,), (1000,), (1000, 50), (1000, 50, 50)]


(True, True, True, True)

## Try just dropping in the LAPACK function directly

In [5]:
def LAPACK_H(states, mu, beta, U, J_matrix, t):
    muf = muc = mu
    eigs = np.empty(shape = states.shape, dtype = np.float64)
    evecs = np.empty(shape = states.shape + (states.shape[-1],), dtype = np.float64)
    e = -t*np.ones(states.shape[-1] - 1)
    ds = U*(states - 1/2) - muc
    
    for i in range(states.shape[0]):
        eigs[i], evecs[i], info = dstev(d = ds[i], e = e, compute_v = True)
    
    return eigs, evecs
%timeit LAPACK_H(states, params.mu, params.beta, params.U, params.J_matrix, params.t)

233 ms ± 68.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Maybe mixing cython into the mix

In [60]:
%%cython --force --annotate
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

import cython
from cython.parallel import prange
cimport numpy as np
import numpy as np
#from ctypes import double

from scipy.linalg.cython_lapack cimport dstev
#void dstev(char *jobz, int *n, d *d, d *e, d *z, int *ldz, d *work, int *info)

## using numpy in cython https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html
## specifics on memory views https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html

@cython.initializedcheck(False)
@cython.infer_types(True)
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef void dstev_vectorised(double[:, ::1] ds, double[:, ::1] es, double[:, :, ::1] evecs, double [::1] work) nogil:
    '''
    Overwrites ds with the eigenvectors and destroys values in ed
    Work needs to be of size 2N - 2
    '''
    cdef int R = ds.shape[0]
    cdef int N = ds.shape[1]
    cdef int info = 0
    cdef int i
    
    for i in prange(R, nogil=True):
        dstev('V', &N, &ds[i][0], &es[i][0], &evecs[i][0][0], &N, &work[0], &info)

ctypedef double[::1] f1D_t      
ctypedef double[:, ::1] f2D_t
ctypedef double[:, :, ::1] f3D_t

#wraparound=False, infer_types=True, initializedcheck=False, cdivision=False
@cython.infer_types(True)
@cython.boundscheck(False)
@cython.initializedcheck(False)
def cython_H(states, mu, beta, U, J_matrix, t):
    cdef int R = states.shape[0]
    cdef int N = states.shape[1]
    
    #allocate the outputs (ds is both input and eigs output) as numpy array but also make views on them
    evecs = np.empty(shape = (R,N,N), dtype = np.float64)
    cdef double[:, :, ::1] evecs_view = evecs
    
    ds = U*(states - 1/2) - mu
    cdef double[:, ::1] ds_view = ds
    
    cdef double[:, ::1] es = -t*np.ones(shape = (R, N - 1), dtype = np.float64)

    cdef double[::1] work = np.empty(shape = 2*N-1, dtype = np.float64)
    cdef int info = 0

    #all the inputs here are typed as contigous memory views, no numpy arrays
    dstev_vectorised(ds_view, es, evecs_view, work)
    
    evals = ds
    
    #the below is a vecorised version of:
    #Ff = - U/2*np.sum(state - 1/2) - muf*np.sum(state) + (state - 1/2).T @ J_matrix @ (state - 1/2)
    #Fc = - 1/beta * np.sum(np.log(1 + np.exp(- beta * evals)))
    S = np.sum(states, axis = -1)
    t = (states - 1/2)
    #shapes(t, J, t) = [(R, N), (N, N), (R, N)] giving contraction ij,ik,ik -> i
    inner_prod = np.einsum('ij,jk,ik -> i', t, J_matrix, t, optimize = 'greedy')
    
    Ff = - U/2*(S - N/2) - mu*S + inner_prod
    Fc = - 1/beta * np.sum(np.log(1 + np.exp(- beta * evals)), axis = -1)
    
    #shapes(Ff, Fc, evals, evecs) = [(R,), (R,), (R, N), (R, N, N)]
    return Ff, Fc, evals, evecs

In [62]:
repeats = 100 #disorder realisations
N = 200 # system size
states = np.random.choice((0.0,1.0), size = (repeats, N))
params = Munch(mu=0.0, beta = 0.5, U = 5.0, t = 1.0, N = N, alpha = 1.5, J = 1.0)
params.J_matrix = interaction_matrix(**params)

nevals = np.array([solve_H_from_FKMC(state, **params)[2] for state in states])
nevecs = np.array([solve_H_from_FKMC(state, **params)[3] for state in states])

print('original')
%time nFf, nFc = np.array([solve_H_from_FKMC(state, **params)[:2] for state in states]).T
print('solve_H_vectorised')
%time Ff, Fc, evals, evecs = solve_H_vectorised(states, **params)
print('cython_H')
%time Ff, Fc, evals, evecs = cython_H(states, params.mu, params.beta, params.U, params.J_matrix, params.t)

shapes(nFc, nFf, Ff, Fc, evals, evecs)

original
CPU times: user 1.7 s, sys: 28 ms, total: 1.73 s
Wall time: 434 ms
solve_H_vectorised
CPU times: user 1.7 s, sys: 20 ms, total: 1.72 s
Wall time: 461 ms
cython_H
CPU times: user 812 ms, sys: 16 ms, total: 828 ms
Wall time: 218 ms
[(100,), (100,), (100,), (100,), (100, 200), (100, 200, 200)]


In [52]:
np.allclose(nevals, evals), np.allclose(nevecs, evecs), np.allclose(Ff, nFf), np.allclose(Fc, nFc)

(True, False, True, True)

In [64]:
from numba import njit, jit, prange

@jit(parallel = True)
def numba_H(states, mu, beta, U, J_matrix, t):
    R, N = states.shape# = (disorder realisations, system size)
    
    muf = muc = mu
    evals = np.empty(shape = states.shape, dtype = np.float64)
    evecs = np.empty(shape = states.shape + (states.shape[-1],), dtype = np.float64)
    e = -t*np.ones(states.shape[-1] - 1)
    ds = U*(states - 1/2) - muc
    
    for i in range(R):
        evals[i], evecs[i] = eigh_tridiagonal(d = ds[i], e = e, lapack_driver = 'stev')
    
    #the below is a vecorised version of:
    #Ff = - U/2*np.sum(state - 1/2) - muf*np.sum(state) + (state - 1/2).T @ J_matrix @ (state - 1/2)
    #Fc = - 1/beta * np.sum(np.log(1 + np.exp(- beta * evals)))
    
    S = np.sum(states, axis = -1)
    t = (states - 1/2)
    #shapes(t, J, t) = [(R, N), (N, N), (R, N)] giving contraction ij,ik,ik -> i
    inner_prod = np.einsum('ij,jk,ik -> i', t, J_matrix, t, optimize = 'greedy')
    
    Ff = - U/2*(S - N/2) - muf*S + inner_prod
    Fc = - 1/beta * np.sum(np.log(1 + np.exp(- beta * evals)), axis = -1)
    
    #shapes(Ff, Fc, evals, evecs) = [(R,), (R,), (R, N), (R, N, N)]
    return Ff, Fc, evals, evecs

numba_H(states, params.mu, params.beta, params.U, params.J_matrix, params.t)
%timeit numba_H(states, params.mu, params.beta, params.U, params.J_matrix, params.t)

435 ms ± 84.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
@njit
def numba_H_2(states, mu, beta, U, J_matrix, t):
    muf = muc = mu
    eigs = np.empty(shape = states.shape, dtype = np.float64)
    evecs = np.empty(shape = states.shape + (states.shape[-1],), dtype = np.float64)
    e = -t*np.ones(states.shape[-1] - 1)
    ds = U*(states - 1/2) - muc
    
    for i in range(states.shape[0]):
        eigs[i], evecs[i] =d
        stev(d = ds[i], e = e, lapack_driver = 'stev')
    
    return eigs, evecs

numba_H_2(states, params.mu, params.beta, params.U, params.J_matrix, params.t)
%timeit numba_H_2(states, params.mu, params.beta, params.U, params.J_matrix, params.t)